In [ ]:
%%file simple_job.py
from mrjob.job import MRJob

# подсчет ДР, с группировкой по первой букве имени

class MRNamesByLetter(MRJob):

    def mapper(self, key, record):
        splits = record.split(",")
        # output: первая буква имени, дата
        yield splits[0][0], int(splits[2])
        

    def reducer(self, letter, births):
        # input: первая буква имени, список дат
        yield letter, sum(births)

if __name__ == '__main__':
    MRNamesByLetter.run()

In [1]:
%%file women_b.py
from mrjob.job import MRJob

# 2 jobs в одном пайплайне
# подсчет женских дней рождения

class MRFemaleNamesByLetter(MRJob):

    def filter_by_gender(self, key, record):
        # выделим только женщик (фильтр)
        splits = record.split(",")
        if splits[1]=="F":
            yield "F", (splits[0]+","+splits[2])

    def count_by_letter(self, key, record):
        splits = record.split(",")
        # output: первая буква имени, дата
        yield splits[0][0], int(splits[1])
        

    def sum_births(self, letter, births):
        # input: первая буква имени, список дат
        yield letter, sum(births)


    def steps(self):
        # создаем пайплайн
        return [self.mr(mapper=self.filter_by_gender),
                self.mr(mapper=self.count_by_letter,
                    reducer=self.sum_births)]


if __name__ == '__main__':
    MRFemaleNamesByLetter.run()

Writing email_connections.py


**Дайте ответ на вопросы:**

    - поучему мы используем yield?
    - обратите внимание на pipeline, что здесь странного?
    - сравните этот вариант с "классической" реалиацией MapReduce, опишите разницу.